# Lab 3.1 - Algorytmy genetyczne
## Dawid Przybyliński 298836

In [1]:
import numpy as np
import random

In [2]:
class evolution:
    
    def __init__(self, population_size, chr_size, target_convert_function, init_method = 'binary'):
        self.n = population_size
        self.m = chr_size
        self.target = target_convert_function
        self.best = None
        
        # initialize random generation
        if init_method == 'binary': # random bits
            self.generation = np.random.choice([0, 1], size=(self.n, self.m), p=[0.5,0.5])
        else: # random uniforms 
            self.generation = np.random.uniform(-8, 8, (self.n, self.m))
            
    
    def evolve(self, p_cross = 0.7, p_mutate = 0.2):
        """
        główna funkcja przeprowadzająca ewolucję i szukająca minimum
        """
        iteration_counter = 0
        current_best_value = -1
        while True:
            
            # check stop condition 
            if iteration_counter == 100:
                return self.target(self.best, arguments = True)
            iteration_counter += 1
            
            # crossover
            parents = self.generation[np.random.choice(self.n, int(np.floor(p_cross*self.n)), replace=False), :]
            crossed = self.crossover(parents)
            population = np.vstack((self.generation, crossed))
            pop_size = population.shape[0]
            
            # mutation
            mutated = population[np.random.choice(pop_size, int(np.floor(p_mutate*pop_size)), replace=False), :]
            mutated = self.mutate(mutated)
            population = np.vstack((population, mutated))
            
            # evaluation
            values = np.asarray([self.target(osobnik) for osobnik in population])
            best_value_index = np.argmax(values)
            if values[best_value_index] > current_best_value:
                self.best = population[best_value_index]
                current_best_value = values[best_value_index]
            
            # selection
            prob = values/values.sum()
            choices = np.random.choice([i for i in range(population.shape[0])], size=self.n, p=prob)
            self.generation = np.asarray([population[choice] for choice in choices])
            
            
            
    def crossover(self,parents):
        """
        krzyżowanie, narazie tylko jednopunktowe
        dla podanych rodziców krzyżujemy 1. z 2., 3. z 4., i tak dalej
        """
        children = []
        for i in range(0, parents.shape[0]-1, 2):
            [child_1, child_2] = self.cross(parents[i], parents[i+1])
            children.append(child_1)
            children.append(child_2)
        return np.asarray(children)
            
            
    def cross(self,x,y):
        """
        krzyżowanie jednopunktowe, punkt podziału jest losowy z przedziału [1,len-1]
        """
        split = random.randint(1,len(x)-1)
        child_1 = np.concatenate((x[0:split], y[split:]))
        child_2 = np.concatenate((y[0:split], x[split:]))
        return [child_1, child_2]
    
    def mutate(self, to_mutate):
        """
        mutacja gaussowska, przesunięcie osobnika o N(0,1)^self.m
        """
        for i, osobnik in enumerate(to_mutate):
            for j, gen in enumerate(osobnik):
                to_mutate[i][j] += np.random.normal(0,1)
        return to_mutate


# Testy
## funkcja kwadratowa w $R^3$: $f(x,y,z) = x^2+y^2+2z^2$

In [3]:
def square_fun(x,y,z):
    """funkcja kwadratowa - pierwsza funckja testowa"""
    return (x**2) + (y**2) + 2*(z**2)

In [4]:
def eval_1(osobnik, arguments = False):
    """przekształcenie funkcji testowej - chcemy szukać minimum funkcji o watościach dodatnich"""
    target = square_fun(osobnik[0],osobnik[1],osobnik[2])
    if arguments:
        return (osobnik,1/target)
    return 1/target   

#### Użyjemy algorytmu 10 razy, wynik jest w formacie $([x,y,z],1/f(x,y,z))$

In [5]:
for i in range(10):
    evo = evolution(100, 3, eval_1, init_method='dupa')
    print(evo.evolve())

(array([-0.02786192, -0.01091604, -0.05909552]), 126.90343142926244)
(array([-0.03461   , -0.00883003, -0.0224529 ]), 437.81179660746557)
(array([ 0.02644371, -0.03369398, -0.02286372]), 347.2157985536121)
(array([ 0.0703555 , -0.00055736,  0.04361346]), 114.2273047906162)
(array([-0.00861182,  0.04241234,  0.01475771]), 433.17226936378114)
(array([0.03533578, 0.0469613 , 0.02738524]), 201.86183800051032)
(array([0.03271546, 0.01320471, 0.02793301]), 356.48433571689634)
(array([ 0.00264797, -0.05572297, -0.04427954]), 142.1784162798045)
(array([-0.0011496 , -0.07625576,  0.02185976]), 147.66772148770133)
(array([ 0.00626394, -0.00059946,  0.01351778]), 2468.787637836067)


Algorytm działa poprawnie, dla populacji o liczności 100 oraz 100 iteracji, rozwiązanie jest blisko minimum globalnego w (0,0,0).

## funkcja Rastrigina w $R^5$: $f(x) = 5A + \sum_{i=1}^5(x_i^2-Acos(\pi x_i))$, $A=10$

In [6]:
def rastrigin(x, A=10):
    """funckja rastrigina"""
    return (A*len(x) + np.sum(x**2 - A*np.cos(2*np.pi*x)))

In [7]:
def eval_2(osobnik, arguments = False):
    target = rastrigin(osobnik)
    if arguments:
        return (osobnik, 1/target)
    return 1/target   

#### Tutaj ponownie przetestujemy algorytm 10 razy, wynik w formacie $(x, 1/f(x))$

In [8]:
for i in range(10):
    evo = evolution(100, 3, eval_2, init_method='dupa')
    print(evo.evolve())

(array([0.00013172, 0.0036699 , 0.00550648]), 115.07203619885341)
(array([ 0.00910999, -0.05781656,  0.00174594]), 1.4858468821286863)
(array([ 0.00209679, -0.07291225, -0.01053509]), 0.9439755537184711)
(array([ 0.02049235, -0.0308745 , -0.04465633]), 1.5042619016761414)
(array([ 0.03745703,  0.04220894, -0.00678303]), 1.5683857386601279)
(array([ 0.03255736,  0.01406953, -0.02439518]), 2.727409863935959)
(array([-0.00179288,  0.02538756, -0.0094037 ]), 6.859832575894723)
(array([ 0.00791516, -0.03128193, -0.00599914]), 4.693002140840459)
(array([-0.00804685,  0.04409351, -0.00882773]), 2.4297026228004976)
(array([-0.03992151, -0.02702822,  0.00836931]), 2.11410119664095)
